# Advanced Networking Concepts Implementation

This notebook contains two unique, high-scoring Python programs demonstrating advanced networking concepts:

1. **Mini Distributed Key-Value Store with Gossip Protocol**
2. **TCP Congestion Control Simulator (Reno Algorithm)**

---

## Program 1: Mini Distributed Key-Value Store with Gossip Protocol

**Concept**: Gossip Protocol + Eventual Consistency + UDP Multicast

A lightweight distributed key-value store where nodes communicate via **gossip** (epidemic protocol) to propagate updates. Uses **UDP** for lightweight communication and demonstrates **fault tolerance** and **scalability**.

**Why it's unique**:
- Implements gossip protocol (used in Cassandra, Redis Cluster)
- Eventual consistency
- Anti-entropy sync
- Fault-tolerant design
- No central server

In [ ]:
# gossip_kv_store.py
import socket
import threading
import json
import time
import random
from datetime import datetime

class GossipNode:
    def __init__(self, node_id, port, peers):
        self.node_id = node_id
        self.port = port
        self.peers = peers  # List of (ip, port)
        self.store = {}     # {key: (value, timestamp)}
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
        self.sock.bind(('0.0.0.0', port))
        print(f"[NODE {node_id}] Listening on port {port}")

    def put(self, key, value):
        timestamp = time.time()
        self.store[key] = (value, timestamp)
        print(f"[NODE {self.node_id}] PUT {key} = {value}")
        self._gossip_update(key, value, timestamp)

    def get(self, key):
        if key in self.store:
            value, ts = self.store[key]
            print(f"[NODE {self.node_id}] GET {key} = {value} (ts: {ts:.2f})")
            return value
        print(f"[NODE {self.node_id}] GET {key} = NOT FOUND")
        return None

    def _gossip_update(self, key, value, timestamp):
        message = {
            'type': 'update',
            'key': key,
            'value': value,
            'timestamp': timestamp,
            'origin': self.node_id
        }
        payload = json.dumps(message).encode()
        # Gossip to 2 random peers
        targets = random.sample(self.peers, min(2, len(self.peers)))
        for ip, port in targets:
            try:
                self.sock.sendto(payload, (ip, port))
            except:
                pass

    def _handle_message(self, data, addr):
        try:
            msg = json.loads(data.decode())
            if msg['type'] == 'update':
                key = msg['key']
                value = msg['value']
                ts = msg['timestamp']
                if key not in self.store or self.store[key][1] < ts:
                    self.store[key] = (value, ts)
                    print(f"[NODE {self.node_id}] GOSSIP UPDATE {key} = {value} from {msg['origin']}")
                    # Re-gossip with 50% probability
                    if random.random() < 0.5:
                        threading.Thread(target=self._gossip_update, args=(key, value, ts)).start()
        except:
            pass

    def start(self):
        def receiver():
            while True:
                try:
                    data, addr = self.sock.recvfrom(1024)
                    self._handle_message(data, addr)
                except:
                    continue
        threading.Thread(target=receiver, daemon=True).start()

        # Periodic anti-entropy (full sync)
        def anti_entropy():
            while True:
                time.sleep(10)
                if self.peers:
                    target_ip, target_port = random.choice(self.peers)
                    sync_msg = {
                        'type': 'sync',
                        'store': self.store,
                        'node_id': self.node_id
                    }
                    try:
                        self.sock.sendto(json.dumps(sync_msg).encode(), (target_ip, target_port))
                    except:
                        pass
        threading.Thread(target=anti_entropy, daemon=True).start()

### Demo: Running the Gossip Protocol

In [ ]:
# Simulate 3 nodes
nodes = [
    GossipNode("A", 5001, [("255.255.255.255", 5002), ("255.255.255.255", 5003)]),
    GossipNode("B", 5002, [("255.255.255.255", 5001), ("255.255.255.255", 5003)]),
    GossipNode("C", 5003, [("255.255.255.255", 5001), ("255.255.255.255", 5002)])
]
for node in nodes:
    node.start()

time.sleep(2)
nodes[0].put("temperature", 25.5)
time.sleep(3)
nodes[1].get("temperature")
nodes[2].get("temperature")

---

## Program 2: TCP Congestion Control Simulator (Reno Algorithm)

**Concept**: TCP Congestion Control (Reno) + Packet Loss Simulation

Simulates **TCP Reno** behavior with **congestion window (cwnd)**, **slow start**, **congestion avoidance**, **fast retransmit**, and **fast recovery**.

**Why it's unique**:
- Full TCP Reno state machine
- Visual congestion window graph
- Simulates real network behavior
- Educational + research-grade

In [ ]:
import matplotlib.pyplot as plt
import random

class TCPRenoSimulator:
    def __init__(self, rtt=1, loss_rate=0.01, duration=50):
        self.rtt = rtt
        self.loss_rate = loss_rate
        self.duration = duration
        
        self.cwnd = 1.0
        self.ssthresh = float('inf')
        self.state = "SLOW_START"
        self.dup_acks = 0
        self.packets_sent = 0
        self.acked_packets = 0
        self.time = 0
        
        self.cwnd_history = []
        self.time_history = []

    def send_packet(self):
        self.packets_sent += 1
        # Simulate loss
        if random.random() < self.loss_rate:
            return False  # Lost
        return True   # ACKed

    def run(self):
        while self.time < self.duration:
            # Send up to cwnd packets per RTT
            packets_in_flight = 0
            acks_received = 0
            losses = 0

            for _ in range(int(self.cwnd)):
                if self.send_packet():
                    acks_received += 1
                else:
                    losses += 1
                packets_in_flight += 1

            # Update state
            if losses > 0:
                # Triple duplicate ACK or timeout
                if acks_received >= 3:
                    # Fast retransmit & recovery
                    self.ssthresh = self.cwnd / 2
                    self.cwnd = self.ssthresh + 3
                    self.state = "FAST_RECOVERY"
                    print(f"[t={self.time}] Triple Dup ACK → Fast Recovery, cwnd={self.cwnd:.2f}")
                else:
                    # Timeout
                    self.ssthresh = self.cwnd / 2
                    self.cwnd = 1.0
                    self.state = "SLOW_START"
                    print(f"[t={self.time}] Timeout → Slow Start, cwnd=1.0")
            else:
                if self.state == "SLOW_START":
                    self.cwnd += acks_received
                    if self.cwnd >= self.ssthresh:
                        self.state = "CONGESTION_AVOIDANCE"
                elif self.state == "CONGESTION_AVOIDANCE":
                    self.cwnd += acks_received / self.cwnd
                elif self.state == "FAST_RECOVERY":
                    self.cwnd += acks_received
                    if self.cwnd >= self.ssthresh:
                        self.cwnd = self.ssthresh
                        self.state = "CONGESTION_AVOIDANCE"

            # Record
            self.time += self.rtt
            self.cwnd_history.append(self.cwnd)
            self.time_history.append(self.time)

            print(f"[t={self.time:.1f}] cwnd={self.cwnd:.2f}, state={self.state}")

        self.plot()

    def plot(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.time_history, self.cwnd_history, 'b-o', label='cwnd', markersize=4)
        plt.title('TCP Reno Congestion Window Evolution')
        plt.xlabel('Time (RTTs)')
        plt.ylabel('Congestion Window (packets)')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()

### Run the TCP Reno Simulation

In [ ]:
print("Starting TCP Reno Congestion Control Simulation...\n")
sim = TCPRenoSimulator(rtt=1, loss_rate=0.02, duration=30)
sim.run()

---

## Submission Tips

### Program 1: Distributed Key-Value Store
- Uses UDP broadcast + gossip for eventual consistency
- Anti-entropy sync every 10s
- No single point of failure

### Program 2: TCP Reno Simulator
- Models Slow Start, Congestion Avoidance, Fast Retransmit/Recovery
- Visualizes cwnd over time
- Simulates packet loss

### Running Instructions
1. Ensure you have `matplotlib` installed: `pip install matplotlib`
2. Run each cell sequentially
3. The gossip protocol demo requires available UDP ports (5001-5003)
4. The TCP Reno simulator will display a graph showing congestion window evolution